In [7]:
# Import necessary packages
import os
import numpy as np
import pylab as py
import matplotlib.pyplot as plt
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import csv

# Paths for isochrones and output
iso_dir = 'isochrones/'
output_dir = 'output_diagrams/'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Estimation variables
star_index = 0      # Set which star in the CSV to analyze
num_top_predictions = 10  # Control how many top predictions to plot

# AKs values from 0 to 1 in increments of 0.1
extinction_values = np.arange(0, 1, 0.1)

# Define static isochrone parameters
dist = 4500
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawCardelli(3.1)
filt_list = ['jwst,F162M', 'jwst,F182M']
filters = ['m_jwst_F162M', 'm_jwst_F182M']
metallicity = 0
level_ages = np.linspace(1, 10, 19) * 1e6  # Define age array
log_age_arr = np.log10(level_ages)

# Load sample magnitudes, skipping the header row
sample_mags = []
with open('../../../../s284-162-182.csv', mode='r') as file:
    csvFile = csv.reader(file)
    next(csvFile)  # Skip header row
    for lines in csvFile:
        sample_mags.append([float(x) for x in lines])

# Chi-square minimization function
def chi_square_reverse_model(iso_grid, sample_mags):
    results = []
    for i, iso in enumerate(iso_grid):
        for star in iso.points:
            chi_square = sum(((sample_mags[k] - star[filters[k]]) ** 2) / star[filters[k]] for k in range(len(sample_mags)))
            results.append([chi_square, star['mass'], 10 ** log_age_arr[i]])
    
    return sorted(results, key=lambda x: x[0])

# Ensure both isochrone and output directories exist before the loop
os.makedirs(iso_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)


# Loop through each extinction value, delete old isochrones, and generate diagrams
for AKs in extinction_values:
    # Clear previous isochrones
    for file in os.listdir(iso_dir):
        os.remove(os.path.join(iso_dir, file))
    
    # Generate isochrone grid
    instances = np.array([
        synthetic.IsochronePhot(log_age, AKs, dist, metallicity=metallicity,
                                evo_model=evo_model, atm_func=atm_func,
                                red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
        for log_age in log_age_arr
    ])

    # Run chi-square minimization and get top results
    sorted_results = chi_square_reverse_model(instances, sample_mags[star_index])
    top_results = sorted_results[:num_top_predictions]

    # Create figure with two subplots side-by-side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    cmap = plt.get_cmap('coolwarm')

    # Color-magnitude diagram (CMD)
    for i, instance in enumerate(instances):
        color = cmap(i / (len(instances) - 1))
        ax1.plot(instance.points[filters[0]] - instance.points[filters[1]], 
                 instance.points[filters[1]], color=color)
    ax1.set_xlabel('F162M - F182M')
    ax1.set_ylabel('F182M')
    ax1.invert_yaxis()
    ax1.grid(True)  # Add grid lines to CMD

    # Magnitude-magnitude diagram (MMD)
    for i, instance in enumerate(instances):
        color = cmap(i / (len(instances) - 1))
        ax2.plot(instance.points[filters[0]], instance.points[filters[1]], color=color)
    ax2.set_xlabel('F162M')
    ax2.set_ylabel('F182M')
    ax2.invert_xaxis()
    ax2.invert_yaxis()
    ax2.grid(True)  # Add grid lines to MMD

    # Create colorbar for both diagrams
    norm = Normalize(vmin=min(level_ages), vmax=max(level_ages))
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=[ax1, ax2])
    cbar.set_label('Age (millions of years)')
    cbar.set_ticks(level_ages)
    cbar.set_ticklabels([f'{age/1e6:.1f}' for age in level_ages])

    # Plot top results on CMD with color gradient and labels
    for j, (chi_square, mass, age) in enumerate(top_results):
        closest_age_idx = np.argmin(abs(level_ages - age))
        iso_instance = instances[closest_age_idx]
        mass_idx = np.argmin(abs(iso_instance.points['mass'] - mass))
        color = cmap(j / (num_top_predictions - 1))
        ax1.plot(iso_instance.points[mass_idx][filters[0]] - iso_instance.points[mass_idx][filters[1]],
                 iso_instance.points[mass_idx][filters[1]], 'o', color=color, label=f"Mass: {mass:.2f}, Age: {age/1e6:.1f} Myr")
        ax2.plot(iso_instance.points[mass_idx][filters[0]],
                 iso_instance.points[mass_idx][filters[1]], 'o', color=color, label=f"Mass: {mass:.2f}, Age: {age/1e6:.1f} Myr")

    # Plot reference star in both diagrams
    ref_star_color = 'gold'  # Optional: distinguish reference star color
    ref_star_marker = '*'  # Optional: distinguish reference star marker

    ax1.plot(sample_mags[star_index][0] - sample_mags[star_index][1], sample_mags[star_index][1], 
         ref_star_marker, markersize=10, color=ref_star_color, label="Reference Star")

    ax2.plot(sample_mags[star_index][0], 
         sample_mags[star_index][1], 
         ref_star_marker, markersize=10, color=ref_star_color, label="Reference Star")

    # Legends and saving
    ax1.legend()
    ax2.legend()
    fig.suptitle(f"AKs = {AKs}")
    plt.savefig(os.path.join(output_dir, f'CMD_MMD_AKs_{AKs}.png'))
    plt.close()

Isochrone generation took 1.218717 s.
Making photometry for isochrone: log(t) = 6.00  AKs = 0.00  dist = 4500
     Starting at:  2024-12-17 01:13:29.986246   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F162M = 21.99
Starting filter: jwst,F182M   Elapsed time: 0.70 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F182M = 22.18
      Time taken: 1.40 seconds
Isochrone generation took 1.128820 s.
Making photometry for isochrone: log(t) = 6.18  AKs = 0.00  dist = 4500
     Starting at:  2024-12-17 01:13:32.534711   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2292 K  m_jwst_F162M = 22.36
Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2292 K  m_jwst_F182M = 22.56
      Time taken: 1.43 seconds
Isochrone genera

      Time taken: 1.40 seconds
Changing to logg=5.00 for T=  1631 logg=4.04
Isochrone generation took 1.176822 s.
Making photometry for isochrone: log(t) = 6.95  AKs = 0.00  dist = 4500
     Starting at:  2024-12-17 01:14:12.229856   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F162M = 25.00
Starting filter: jwst,F182M   Elapsed time: 0.70 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F182M = 24.84
      Time taken: 1.40 seconds
Changing to logg=5.00 for T=  1602 logg=4.04
Isochrone generation took 1.165268 s.
Making photometry for isochrone: log(t) = 6.98  AKs = 0.00  dist = 4500
     Starting at:  2024-12-17 01:14:14.810158   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F162M = 25.15
Starting filter: jwst,F182M   Elapsed time: 0.69 seconds
Starting

      Time taken: 1.36 seconds
Changing to logg=5.00 for T=  1773 logg=4.01
Isochrone generation took 1.222845 s.
Making photometry for isochrone: log(t) = 6.88  AKs = 0.10  dist = 4500
     Starting at:  2024-12-17 01:14:55.316098   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1773 K  m_jwst_F162M = 24.45
Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1773 K  m_jwst_F182M = 24.52
      Time taken: 1.40 seconds
Changing to logg=5.00 for T=  1711 logg=4.02
Isochrone generation took 1.167420 s.
Making photometry for isochrone: log(t) = 6.90  AKs = 0.10  dist = 4500
     Starting at:  2024-12-17 01:14:57.896450   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1711 K  m_jwst_F162M = 24.76
Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting

      Time taken: 1.40 seconds
Changing to logg=5.00 for T=  1870 logg=3.98
Isochrone generation took 1.236286 s.
Making photometry for isochrone: log(t) = 6.78  AKs = 0.20  dist = 4500
     Starting at:  2024-12-17 01:15:38.457019   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1870 K  m_jwst_F162M = 24.09
Starting filter: jwst,F182M   Elapsed time: 0.76 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1870 K  m_jwst_F182M = 24.39
      Time taken: 1.52 seconds
Changing to logg=5.00 for T=  1838 logg=3.99
Isochrone generation took 1.239255 s.
Making photometry for isochrone: log(t) = 6.81  AKs = 0.20  dist = 4500
     Starting at:  2024-12-17 01:15:41.227430   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1838 K  m_jwst_F162M = 24.25
Starting filter: jwst,F182M   Elapsed time: 0.74 seconds
Starting

      Time taken: 1.46 seconds
Isochrone generation took 1.271260 s.
Making photometry for isochrone: log(t) = 6.65  AKs = 0.30  dist = 4500
     Starting at:  2024-12-17 01:16:22.069102   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2011 K  m_jwst_F162M = 23.93
Starting filter: jwst,F182M   Elapsed time: 0.68 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2011 K  m_jwst_F182M = 24.14
      Time taken: 1.37 seconds
Changing to logg=5.00 for T=  1960 logg=3.95
Isochrone generation took 1.204485 s.
Making photometry for isochrone: log(t) = 6.70  AKs = 0.30  dist = 4500
     Starting at:  2024-12-17 01:16:24.656216   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1960 K  m_jwst_F162M = 23.98
Starting filter: jwst,F182M   Elapsed time: 0.70 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  

...

Starting filter: jwst,F182M   Elapsed time: 0.70 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F182M = 25.49
      Time taken: 1.41 seconds
Changing to logg=5.00 for T=  1602 logg=4.04
Isochrone generation took 1.199793 s.
Making photometry for isochrone: log(t) = 6.98  AKs = 0.50  dist = 4500
     Starting at:  2024-12-17 01:18:31.540008   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F162M = 25.94
Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F182M = 25.61
      Time taken: 1.40 seconds
Changing to logg=5.00 for T=  1571 logg=4.05
Isochrone generation took 1.314643 s.
Making photometry for isochrone: log(t) = 7.00  AKs = 0.50  dist = 4500
     Starting at:  2024-12-17 01:18:34.269122   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting

Starting filter: jwst,F182M   Elapsed time: 0.90 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1773 K  m_jwst_F182M = 25.18
      Time taken: 1.80 seconds
Changing to logg=5.00 for T=  1711 logg=4.02
Isochrone generation took 1.242682 s.
Making photometry for isochrone: log(t) = 6.90  AKs = 0.60  dist = 4500
     Starting at:  2024-12-17 01:19:20.296441   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1711 K  m_jwst_F162M = 25.56
Starting filter: jwst,F182M   Elapsed time: 0.91 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1711 K  m_jwst_F182M = 25.35
      Time taken: 1.81 seconds
Changing to logg=5.00 for T=  1677 logg=4.03
Isochrone generation took 1.422212 s.
Making photometry for isochrone: log(t) = 6.93  AKs = 0.60  dist = 4500
     Starting at:  2024-12-17 01:19:23.552805   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting

Starting filter: jwst,F182M   Elapsed time: 0.72 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1870 K  m_jwst_F182M = 25.05
      Time taken: 1.41 seconds
Changing to logg=5.00 for T=  1838 logg=3.99
Isochrone generation took 1.246840 s.
Making photometry for isochrone: log(t) = 6.81  AKs = 0.70  dist = 4500
     Starting at:  2024-12-17 01:20:08.005811   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1838 K  m_jwst_F162M = 25.05
Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1838 K  m_jwst_F182M = 25.13
      Time taken: 1.41 seconds
Changing to logg=5.00 for T=  1814 logg=4.00
Isochrone generation took 1.241089 s.
Making photometry for isochrone: log(t) = 6.85  AKs = 0.70  dist = 4500
     Starting at:  2024-12-17 01:20:10.670425   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting

Starting filter: jwst,F182M   Elapsed time: 0.72 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2011 K  m_jwst_F182M = 24.80
      Time taken: 1.44 seconds
Changing to logg=5.00 for T=  1960 logg=3.95
Isochrone generation took 1.269396 s.
Making photometry for isochrone: log(t) = 6.70  AKs = 0.80  dist = 4500
     Starting at:  2024-12-17 01:20:51.513579   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1960 K  m_jwst_F162M = 24.78
Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1960 K  m_jwst_F182M = 24.94
      Time taken: 1.42 seconds
Changing to logg=5.00 for T=  1893 logg=3.97
Isochrone generation took 1.234112 s.
Making photometry for isochrone: log(t) = 6.74  AKs = 0.80  dist = 4500
     Starting at:  2024-12-17 01:20:54.187354   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting

Starting filter: jwst,F182M   Elapsed time: 0.78 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2143 K  m_jwst_F182M = 24.45
      Time taken: 1.50 seconds
Isochrone generation took 1.257622 s.
Making photometry for isochrone: log(t) = 6.54  AKs = 0.90  dist = 4500
     Starting at:  2024-12-17 01:21:35.905381   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2099 K  m_jwst_F162M = 24.59
Starting filter: jwst,F182M   Elapsed time: 0.72 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2099 K  m_jwst_F182M = 24.67
      Time taken: 1.43 seconds
Isochrone generation took 1.288085 s.
Making photometry for isochrone: log(t) = 6.60  AKs = 0.90  dist = 4500
     Starting at:  2024-12-17 01:21:38.643609   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2052 K  m_jwst_F162M = 24.74
Starting filter: